In [19]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, PrefixTuningConfig, PromptEncoderConfig, IA3Config, PeftModel
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
import datetime
import os

In [2]:
def format_time(seconds):
    return str(datetime.timedelta(seconds=int(seconds)))

In [3]:
imdb = load_dataset("imdb")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    "HamsterShiu/BERT_MLM", 
    subfolder="checkpoint-95000"
)

In [5]:
tokenized_imdb = imdb.map(lambda e: tokenizer(e["text"], truncation=True, padding=True), batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
tokenized_imdb = tokenized_imdb.remove_columns(["text"])
tokenized_imdb = tokenized_imdb.rename_column("label", "labels")
tokenized_imdb.set_format("torch")

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [10]:
peft_methods = {
    "LoRA": LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["query", "key", "value"],
        bias="none",
        inference_mode=False,
    ),
    "Prefix Tuning": PrefixTuningConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=20,
        prefix_projection=True,
        encoder_hidden_size=768,
    ),
    "IA³": IA3Config(
        task_type=TaskType.SEQ_CLS,
        target_modules=["query", "key", "value", "output.dense"],
        feedforward_modules=["output.dense"],
        inference_mode=False,
    )
}

In [22]:
for method_name, peft_config in peft_methods.items():
    print(f"\n{'='*50}")
    print(f"Training with {method_name}")
    print(f"{'='*50}")
    
    checkpoint_path = f"./SC4001/Assignment2/model/peft_bert_{method_name.lower().replace(' ', '_')}_final"
    
    if os.path.exists(checkpoint_path):
        print(f"Finetuned model exists: {checkpoint_path}")
        
        base_model = AutoModelForSequenceClassification.from_pretrained(
            "HamsterShiu/BERT_MLM",
            subfolder="checkpoint-95000",
            num_labels=2
        )
        model = PeftModel.from_pretrained(base_model, checkpoint_path)
        
        # Use pre-calculated stats if available (you'd need to save this info separately)
        if method_name in results:
            print(f"Using existing results for {method_name}")
        else:
            # Evaluate the model
            training_args = TrainingArguments(
                output_dir=f"./SC4001/Assignment2/model/peft_bert_{method_name.lower().replace(' ', '_')}_eval",
                per_device_eval_batch_size=16,
                report_to="none",
            )
            
            trainer = Trainer(
                model=model,
                args=training_args,
                eval_dataset=tokenized_imdb["test"],
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics,
            )
            
            eval_results = trainer.evaluate()
            
            # Count trainable parameters
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            total_params = sum(p.numel() for p in model.parameters())
            
            # Add to results without training time (since it was pre-trained)
            results[method_name] = {
                "trainable_params": trainable_params,
                "trainable_params_percentage": trainable_params/total_params*100,
                "eval_results": eval_results
            }
            
    else:
        print("Finetuned model does not exist. Finetuning now.")
        
        # Load a fresh base model for each method
        base_model = AutoModelForSequenceClassification.from_pretrained(
            "HamsterShiu/BERT_MLM",
            subfolder="checkpoint-95000",
            num_labels=2
        )
        
        # Apply the PEFT configuration
        model = get_peft_model(base_model, peft_config)
        
        # Print trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Trainable parameters: {trainable_params} ({trainable_params/total_params*100:.2f}%)")
        
        # Training arguments
        training_args = TrainingArguments(
            output_dir=f"./SC4001/Assignment2/model/peft_bert_{method_name.lower().replace(' ', '_')}",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=2,
            weight_decay=0.01,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            push_to_hub=False,
            logging_steps=100,
        )
        
        # Initialize Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_imdb["train"],
            eval_dataset=tokenized_imdb["test"],
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )
        
        # Measure training time
        start_time = time.time()
        
        # Train model
        train_result = trainer.train()
        
        # Calculate training time
        training_time = time.time() - start_time
        
        # Evaluate model
        eval_results = trainer.evaluate()
        
        # Save results
        results[method_name] = {
            "training_time": training_time,
            "training_time_formatted": format_time(training_time),
            "trainable_params": trainable_params,
            "trainable_params_percentage": trainable_params/total_params*100,
            "eval_results": eval_results
        }
        
        # Save PEFT model
        model.save_pretrained(checkpoint_path)
        print(f"Saved model to {checkpoint_path}")
        
        print(f"\nTraining time: {format_time(training_time)}")
        print(f"Evaluation results: {eval_results}")


Training with LoRA
Finetuned model exists: ./SC4001/Assignment2/model/peft_bert_lora_final


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HamsterShiu/BERT_MLM and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_4010346/3146117098.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Training with Prefix Tuning
Finetuned model exists: ./SC4001/Assignment2/model/peft_bert_prefix_tuning_final


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HamsterShiu/BERT_MLM and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_4010346/3146117098.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Training with IA³
Finetuned model exists: ./SC4001/Assignment2/model/peft_bert_ia³_final


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HamsterShiu/BERT_MLM and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_4010346/3146117098.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
# Print comparison of all methods
print("\n" + "="*80)
print("PEFT Methods Comparison")
print("="*80)
print(f"{'Method':<15} {'Training Time':<15} {'Params %':<10} {'Accuracy':<10} {'F1':<10}")
print("-"*80)

for method, data in results.items():
    # For pre-trained models that don't have training time recorded
    training_time = data.get("training_time_formatted", "Pre-trained")
    
    print(f"{method:<15} {training_time:<15} {data['trainable_params_percentage']:.2f}%{' ':<5} {data['eval_results']['eval_accuracy']:.4f}{' ':<5} {data['eval_results']['eval_f1']:.4f}")


PEFT Methods Comparison
Method          Training Time   Params %   Accuracy   F1        
--------------------------------------------------------------------------------
LoRA            Pre-trained     0.00%      0.5743      0.3021
Prefix Tuning   Pre-trained     0.00%      0.9011      0.9017
IA³             Pre-trained     0.00%      0.9056      0.9062
